In [1]:
import os
import re
import sys
import json
import unicodedata
import pandas as pd
import psycopg2 as pg2
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from pymongo import MongoClient

from src.run import hot_soup, price_to_float
from src.crawler import Crawler

client = MongoClient('192.168.0.209', 27017)
db = client['reverb']
link_coll = db['links']
sales_coll = db['sales']
data_coll = db['data']

In [2]:
brands = [
    'Airline',
    'American Showster',
    'Ampeg',
    'Aria',
    'B.C. Rich',
    'B3',
    'BC Rich',
    'Baldwin',
    'Bilt',
    'Bogner',
    'Burns',
    'Campbell',
    'Charvel',
    'Collings',
    'Conrad',
    'Cort',
    "D'Angelico",
    'Danelectro',
    'DeArmond',
    'Dean',
    'DiPinto',
    'Dobro',
    'Duesenberg',
    'EKO',
    'ESP',
    'EVH',
    'Eastman',
    'Eastwood',
    'Electra',
    'Epiphone',
    'Ernie Ball Music Man',
    'Fano',
    'Fender',
    'Framus',
    'G&L',
    'Gibson',
    'Giffin',
    'Godin',
    'Goya',
    'Gretsch',
    'Grosh',
    'Guild',
    'Guyatone',
    'Hagstrom',
    'Hamer',
    'Harden Engineering',
    'Harmony',
    'Heritage',
    'Hofner',
    'Hohner',
    'Ibanez',
    'Intermark',
    'Jackson',
    'James Trussart',
    'Jerry Jones',
    'Kalamazoo',
    'Kay',
    'Kimberly',
    'Kramer',
    'La Baye',
    'Larrivee',
    'Line 6',
    'MCI',
    'Magnatone',
    'Martin',
    'Memphis',
    'Micro-Frets',
    'Moog',
    'Mosrite',
    'Music Man',
    'Musicraft',
    'Musicvox',
    'Nash',
    'National',
    'Norma',
    'Orville',
    'Ovation',
    'Parker',
    'Paul Reed Smith',
    'Peavey',
    'Premier',
    'Reverend',
    'Rickenbacker',
    'Roland',
    'SWR',
    'Schecter',
    'Silvertone',
    'Sorrento',
    'Squier',
    'Steinberger',
    'Suhr',
    'Supro',
    'Taylor',
    'Teisco',
    'The Loar',
    'Tokai',
    'Tom Anderson',
    'Travis',
    'Truetone',
    'Two Rock',
    'Univox',
    'Vox',
    'Washburn',
    'Yamaha'
]

In [3]:
sales_df = pd.DataFrame(list(sales_coll.find({},{'_id':0})))
print(sales_df.info())
sales_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145601 entries, 0 to 145600
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   title   145601 non-null  object 
 1   date    145601 non-null  object 
 2   cond    145601 non-null  object 
 3   price   145601 non-null  float64
dtypes: float64(1), object(3)
memory usage: 4.4+ MB
None


,title,date,cond,price
0,Rickenbacker 660-12 Tom Petty Signature 1992,2/23/2021,Excellent,6495.0
1,Rickenbacker 660-12 Tom Petty Signature 1992,12/7/2020,Excellent,4325.0
2,Rickenbacker 660-12 Tom Petty Signature 1992,10/20/2020,Excellent,7995.0
3,Rickenbacker 660-12 Tom Petty Signature 1992,9/28/2020,Excellent,5495.0
4,Rickenbacker 660-12 Tom Petty Signature 1992,5/26/2020,Excellent,5800.0


In [4]:
links_df = pd.DataFrame(list(link_coll.find({}, {'_id': 0})))
links_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5065 entries, 0 to 5064
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   5065 non-null   object
 1   link    5065 non-null   object
 2   html    2994 non-null   object
dtypes: object(3)
memory usage: 118.8+ KB


In [5]:
data_df = pd.DataFrame(list(data_coll.find({}, {'_id': 0})))
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728 entries, 0 to 727
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   title     728 non-null    object 
 1   brand     728 non-null    object 
 2   model     728 non-null    object 
 3   year      728 non-null    object 
 4   color     727 non-null    object 
 5   est_low   728 non-null    float64
 6   est_high  728 non-null    float64
 7   text      632 non-null    object 
dtypes: float64(2), object(6)
memory usage: 45.6+ KB


In [6]:
for link in list(link_coll.find({'html': {'$exists': True}}))[:10]:
    print(link['link'])

https://reverb.com/price-guide/guide/31-rickenbacker-660-12-tom-petty-signature-1992
https://reverb.com/price-guide/guide/33-fender-coronado-ii-1968-antigua
https://reverb.com/price-guide/guide/38-fender-stratocaster-1958-sunburst
https://reverb.com/price-guide/guide/42-fender-stratocaster-1964-sunburst
https://reverb.com/price-guide/guide/124-epiphone-zephyr-1944-sunburst
https://reverb.com/price-guide/guide/677-fender-esquire-1951-butterscotch-blonde
https://reverb.com/price-guide/guide/685-fender-jazzmaster-1960-sunburst
https://reverb.com/price-guide/guide/684-fender-jazzmaster-1959-sunburst-tortoise-pickguard
https://reverb.com/price-guide/guide/683-fender-jazzmaster-1959-sunburst-with-gold-pickguard
https://reverb.com/price-guide/guide/689-fender-jazzmaster-1962-3-tone-sunburst


In [7]:
'|'.join(brands)

"Airline|American Showster|Ampeg|Aria|B.C. Rich|B3|BC Rich|Baldwin|Bilt|Bogner|Burns|Campbell|Charvel|Collings|Conrad|Cort|D'Angelico|Danelectro|DeArmond|Dean|DiPinto|Dobro|Duesenberg|EKO|ESP|EVH|Eastman|Eastwood|Electra|Epiphone|Ernie Ball Music Man|Fano|Fender|Framus|G&L|Gibson|Giffin|Godin|Goya|Gretsch|Grosh|Guild|Guyatone|Hagstrom|Hamer|Harden Engineering|Harmony|Heritage|Hofner|Hohner|Ibanez|Intermark|Jackson|James Trussart|Jerry Jones|Kalamazoo|Kay|Kimberly|Kramer|La Baye|Larrivee|Line 6|MCI|Magnatone|Martin|Memphis|Micro-Frets|Moog|Mosrite|Music Man|Musicraft|Musicvox|Nash|National|Norma|Orville|Ovation|Parker|Paul Reed Smith|Peavey|Premier|Reverend|Rickenbacker|Roland|SWR|Schecter|Silvertone|Sorrento|Squier|Steinberger|Suhr|Supro|Taylor|Teisco|The Loar|Tokai|Tom Anderson|Travis|Truetone|Two Rock|Univox|Vox|Washburn|Yamaha"

In [9]:
# This works 100% here but fails on the title "Kalamazoo KG-2a 1960s Blue" in a terminal-run python script. Not sure why

regex = ")(?:(?:(.+)((?:Early|Mid|Late)-?\s?'?\d+s|" \
        "\d{4}\s?-\s?\d{4})(.*))|" \
        "(?:(.+)(\d{4}s?)(.*))|" \
        "(?:(.+)(\d{2}s?)(.*)))$"
regex = r"(" + '|'.join(brands) + regex

docs = list(link_coll.find({'html': {'$exists': True}}))

for document in tqdm(docs):
    title = document['title']
    html = document.pop('html')
    new_doc = {'title': title}
    
    # Parse Title
    groups = re.match(regex, title, re.IGNORECASE).groups()
    feats = [g.strip() if g else None for g in groups if g is not None]
    new_doc.update({k: feats[i] for i, k in enumerate(['brand', 'model', 'year', 'color'])})
    
    # Scrape html
    soup = hot_soup(html)
    
    reverb_price_range = soup.find_all('span', class_='used')
    est_low, est_high = (price_to_float(s.text) for s in reverb_price_range)
    new_doc['est_low'] = est_low
    new_doc['est_high'] = est_high
    
    text_section = soup.find('section', class_='scaling-pb-2')
    if text_section:
        text = text_section.p.get_text(separator='|')
        text = unicodedata.normalize('NFKD', text)
        new_doc['text'] = text
    else:
        new_doc['text'] = None
        
    data_coll.insert_one(new_doc)

 55%|█████▍    | 1644/2994 [00:48<00:39, 33.93it/s]


ValueError: not enough values to unpack (expected 2, got 0)

In [11]:
# conn = pg2.connect(dbname='reverb', host='192.168.0.209', password='galvanize', user='postgres')
# cur = conn.cursor()
# conn.autocommit = True
# cur.execute("""CREATE TABLE guitars(
#                id INT PRIMARY KEY GENERATED ALWAYS AS IDENTITY,
#                title VARCHAR(225),
#                date_str VARCHAR(12),
#                cond VARCHAR(12),
#                sale_price NUMERIC
#            );""")
# conn.close()